In [25]:
# Laboratorio 3
### Roberto Rios 20979
### Javier Mombiela 20067

In [26]:
#importando librerias
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, BatchNormalization

In [27]:
# cagamos el dataset
data = pd.read_csv('sunspots.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Month     2820 non-null   object 
 1   Sunspots  2820 non-null   float64
dtypes: float64(1), object(1)
memory usage: 44.2+ KB


In [28]:
#imprimimos la data
data

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
...,...,...
2815,1983-08,71.8
2816,1983-09,50.3
2817,1983-10,55.8
2818,1983-11,33.3


In [29]:
# separamos 'Month' en dos columnas, 'Year' y 'Month'
data[['Year','MonthNum']] = data['Month'].str.split('-',expand=True)

# print(data['Year'], data['MonthNum'])
X = data[['Year','MonthNum']]
y = data['Sunspots']

# dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:
# creamos la red feed forward
feedForward = Sequential([
    Dense(64, input_shape=(2,), activation="relu"),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='linear')
])

# compilamos la red
feedForward.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [31]:
# entrenamos la red
history1 = feedForward.fit(
    X_train_scaled, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    verbose=2
)

Epoch 1/100
71/71 - 2s - loss: 4378.7935 - mse: 4378.7935 - 2s/epoch - 29ms/step
Epoch 2/100
71/71 - 0s - loss: 4181.3545 - mse: 4181.3545 - 143ms/epoch - 2ms/step
Epoch 3/100
71/71 - 0s - loss: 3957.4216 - mse: 3957.4216 - 130ms/epoch - 2ms/step
Epoch 4/100
71/71 - 0s - loss: 3677.4729 - mse: 3677.4729 - 130ms/epoch - 2ms/step
Epoch 5/100
71/71 - 0s - loss: 3343.2258 - mse: 3343.2258 - 138ms/epoch - 2ms/step
Epoch 6/100
71/71 - 0s - loss: 2971.7268 - mse: 2971.7268 - 154ms/epoch - 2ms/step
Epoch 7/100
71/71 - 0s - loss: 2576.4263 - mse: 2576.4263 - 154ms/epoch - 2ms/step
Epoch 8/100
71/71 - 0s - loss: 2257.0112 - mse: 2257.0112 - 143ms/epoch - 2ms/step
Epoch 9/100
71/71 - 0s - loss: 2014.1870 - mse: 2014.1870 - 152ms/epoch - 2ms/step
Epoch 10/100
71/71 - 0s - loss: 1836.1554 - mse: 1836.1554 - 154ms/epoch - 2ms/step
Epoch 11/100
71/71 - 0s - loss: 1732.7283 - mse: 1732.7283 - 155ms/epoch - 2ms/step
Epoch 12/100
71/71 - 0s - loss: 1669.0607 - mse: 1669.0607 - 154ms/epoch - 2ms/step
Epo

In [32]:
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

plot_metric(history1, 'mse')
plot_metric(history1, 'loss')

feedForward.evaluate(X_test_scaled, y_test, verbose=2)


KeyError: 'val_mse'